## Calibrating Trip Mode Choice
This script will iteratively perform updates to the trip mode choice coefficients config file in order to match model outputs to calibration targets.

### Load Libraries

In [1]:
import pandas as pd
import numpy as np
import os
# import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

from IPython.display import display
import importlib
import asim_trip_calib_util
importlib.reload(asim_trip_calib_util)
# from asim_trip_calib_util import *
# check to make sure the dictionaries specifying names for calibration targets and activitysim outputs are consistent
asim_trip_calib_util.check_input_dictionaries_for_consistency()

No problems found in input dictionaries


### Required Inputs
This script requires a working version of ActivitySim installed in the conda environment.  Input data sources are:
* Initial model output directory that includes household, tour and trip files
* trip mode choice calibration target tables. Tables should be indexed by trip mode and columns should be tour mode.  Tables should be broken down by tour purpose
* Model config directory containing the trip mode choice coefficients

Changes in tour, trip modes and purposes can be implemented by changing the dictionaries at the top of asim_trip_calib_util_CMAP.py

In [2]:
# csv file containing calibration targets by tour mode choice.
# Column names and acceptable values should be set in dict at top of script
trip_mode_choice_calib_targets_file = r"C:\abm_runs\rohans\calibration\trip_mc\targets\trip_mode_choice_calibration_targets_2024-01-23_updated.csv"

# directory of the simulation.py file
simpy_dir = r"C:\abm_runs\rohans"

# location of configuration files
settings_dir = r"C:\abm_runs\rohans\configs\resident\settings_mp.yaml"
configs_resident_dir = r"C:\abm_runs\rohans\configs\resident"
configs_common_dir = r"C:\abm_runs\rohans\configs\common"

warm_start_settings_mp_file = r"C:\abm_runs\rohans\calibration\trip_mc\scripts\settings_mp_warm_start.yaml"
cold_start_settings_mp_file = r"C:\abm_runs\rohans\calibration\trip_mc\scripts\settings_mp_cold_start.yaml"
trip_mc_coef_file = None 

# input data location
data_dir = r"C:\abm_runs\rohans\input_2022"

# output location
output_dir = r"C:\abm_runs\rohans\calibration\trip_mc\output\calibration_output_an_iter_cold"
activitysim_run_dir = r"C:\abm_runs\rohans\calibration\trip_mc\output\activitysim_run_dir"

# calibration iterations
calibration_iterations_to_run = 5

# want to do intial model run first?
want_to_do_initial_model_run = True  # True or False

# calibration settings
max_ASC_adjust = 5 # maximum allowed adjustment per iteration
damping_factor = 1 # constant multiplied to all adjustments
adjust_when_zero_counts = 2 # coefficient change when have target counts but no model counts (or vise-versa)


--------------- User should not have to change anything below this line ----------------------

## Initial Model Run

In [3]:
### Change directory to model setup
### i.e. the location of simulation.py script
os.chdir(simpy_dir)

In [9]:
if want_to_do_initial_model_run:
    asim_trip_calib_util.run_activitysim(
        data_dir=data_dir,  # data inputs for ActivitySim
        configs_resident_dir=configs_resident_dir,  # these files are copied to the config section of the run directory
        configs_common_dir=configs_common_dir,  # these files are copied to the config section of the run directory
        run_dir=activitysim_run_dir,  # ActivitySim run directory
        output_dir=output_dir,  # location to store run model outputs
        settings_file=cold_start_settings_mp_file,  # optional: ActivitySim settings.yaml to replace the one in configs_dir
        trip_mc_coef_file=trip_mc_coef_file  # optional: trip_mode_choice_coefficients.csv to replace the one in configs_dir
    )
    
    # _ = asim_trip_calib_util.perform_trip_mode_choice_model_calibration(
    #     asim_output_dir=output_dir, # folder containing the activitysim model output
    #     asim_configs_dir=os.path.join(activitysim_run_dir, 'configs'),  # folder containing activitysim trip mode choice config files
    #     trip_mode_choice_calib_targets_file=trip_mode_choice_calib_targets_file,  # folder containing trip mode choice calibration tables
    #     max_ASC_adjust=max_ASC_adjust,  
    #     damping_factor=damping_factor,  # constant multiplied to all adjustments
    #     adjust_when_zero_counts=adjust_when_zero_counts,
    #     output_dir=output_dir,  # location to write model calibration steps
    # )
    # trip_mc_coef_file = os.path.join(output_dir, 'trip_mode_choice_coefficients.csv') 
else:
    print("No initial model run performed.")
    

ActivitySim run started at:  2024-05-16 14:45:16.374744
python simulation.py -s C:\abm_runs\rohans\calibration\trip_mc\scripts\settings_mp_cold_start.yaml -c C:\abm_runs\rohans\calibration\trip_mc\output\activitysim_run_dir\configs -c C:\abm_runs\rohans\configs\common -d C:\abm_runs\rohans\input_2022 -o C:\abm_runs\rohans\calibration\trip_mc\output\activitysim_run_dir
ActivitySim ended at 2024-05-16 19:11:18.495818
Run Time:  15962.12 secs =  266.03533333333337  mins


In [10]:
_ = asim_trip_calib_util.perform_trip_mode_choice_model_calibration(
        asim_output_dir=output_dir, # folder containing the activitysim model output
        asim_configs_dir=os.path.join(activitysim_run_dir, 'configs'),  # folder containing activitysim trip mode choice config files
        trip_mode_choice_calib_targets_file=trip_mode_choice_calib_targets_file,  # folder containing trip mode choice calibration tables
        max_ASC_adjust=max_ASC_adjust,  
        damping_factor=damping_factor,  # constant multiplied to all adjustments
        adjust_when_zero_counts=adjust_when_zero_counts,
        output_dir=output_dir,  # location to write model calibration steps
    )

Sample rate of  1 results in  12523739 out of 12523739 tours


FileNotFoundError: [Errno 2] No such file or directory: 'C:\\abm_runs\\rohans\\calibration\\trip_mc\\output\\activitysim_run_dir\\configs\\trip_mode_choice_coefficients.csv'

## Iterating

In [9]:
iteration_output_dir = output_dir.strip('_cold') + '_10'

calibration_iterations_to_run = 3
start_iter_num = 1

for i in range(start_iter_num, calibration_iterations_to_run+start_iter_num):
    asim_trip_calib_util.run_activitysim(
        data_dir=data_dir,  # data inputs for ActivitySim
        configs_resident_dir=configs_resident_dir,  # these files are copied to the config section of the run directory
        configs_common_dir=configs_common_dir,  # these files are copied to the config section of the run directory
        run_dir=activitysim_run_dir,  # ActivitySim run directory
        output_dir=iteration_output_dir,  # location to store run model outputs
        settings_file=warm_start_settings_mp_file,  # optional: ActivitySim settings.yaml to replace the one in configs_dir
        trip_mc_coef_file=trip_mc_coef_file  # optional: trip_mode_choice_coefficients.csv to replace the one in configs_dir
    )
    
    _ = asim_trip_calib_util.perform_trip_mode_choice_model_calibration(
        asim_output_dir=iteration_output_dir, # folder containing the activitysim model output
        asim_configs_dir=os.path.join(activitysim_run_dir, 'configs'),  # folder containing activitysim trip mode choice config files
        trip_mode_choice_calib_targets_file=trip_mode_choice_calib_targets_file,  # folder containing trip mode choice calibration tables
        max_ASC_adjust=max_ASC_adjust,  
        damping_factor=damping_factor,  # constant multiplied to all adjustments
        adjust_when_zero_counts=adjust_when_zero_counts,
        output_dir=iteration_output_dir,  # location to write model calibration steps
    )
    trip_mc_coef_file = os.path.join(iteration_output_dir, 'trip_mode_choice_coefficients.csv') 
    iteration_output_dir = iteration_output_dir.strip('_'+str(i)) + '_' + str(i+1)

print("\n\n", "Final coefficient table written to: ", trip_mc_coef_file)

ActivitySim run started at:  2024-05-14 12:13:38.539136
python simulation.py -s C:\abm_runs\rohans\calibration\trip_mc\scripts\settings_mp_warm_start.yaml -c C:\abm_runs\rohans\calibration\trip_mc\output\activitysim_run_dir\configs -c C:\abm_runs\rohans\configs\common -d C:\abm_runs\rohans\input_2022 -o C:\abm_runs\rohans\calibration\trip_mc\output\activitysim_run_dir
ActivitySim ended at 2024-05-14 16:14:59.984177
Run Time:  14481.45 secs =  241.35750000000002  mins


AssertionError: ActivitySim run not completed! See ActivitySim log file for details.

In [ ]:
# trip_mc_coef_file = r'C:\abm_runs\rohans\calibration\trip_mc\output\calibration_output_an_iter_3\trip_mode_choice_coefficients.csv'
# iteration_output_dir = output_dir.strip('_cold') + '_4'

# print(trip_mc_coef_file)
# print(iteration_output_dir)

In [ ]:
# calibration_iterations_to_run = 3
# start_iter_num = 4

# for i in range(start_iter_num, calibration_iterations_to_run+start_iter_num):
#     asim_trip_calib_util.run_activitysim(
#         data_dir=data_dir,  # data inputs for ActivitySim
#         configs_resident_dir=configs_resident_dir,  # these files are copied to the config section of the run directory
#         configs_common_dir=configs_common_dir,  # these files are copied to the config section of the run directory
#         run_dir=activitysim_run_dir,  # ActivitySim run directory
#         output_dir=iteration_output_dir,  # location to store run model outputs
#         settings_file=warm_start_settings_mp_file,  # optional: ActivitySim settings.yaml to replace the one in configs_dir
#         trip_mc_coef_file=trip_mc_coef_file  # optional: trip_mode_choice_coefficients.csv to replace the one in configs_dir
#     )
    
#     _ = asim_trip_calib_util.perform_trip_mode_choice_model_calibration(
#         asim_output_dir=iteration_output_dir, # folder containing the activitysim model output
#         asim_configs_dir=os.path.join(activitysim_run_dir, 'configs'),  # folder containing activitysim trip mode choice config files
#         trip_mode_choice_calib_targets_file=trip_mode_choice_calib_targets_file,  # folder containing trip mode choice calibration tables
#         max_ASC_adjust=max_ASC_adjust,  
#         damping_factor=damping_factor,  # constant multiplied to all adjustments
#         adjust_when_zero_counts=adjust_when_zero_counts,
#         output_dir=iteration_output_dir,  # location to write model calibration steps
#     )
#     trip_mc_coef_file = os.path.join(iteration_output_dir, 'trip_mode_choice_coefficients.csv') 
#     iteration_output_dir = iteration_output_dir.strip('_'+str(i)) + '_' + str(i+1)

# print("\n\n", "Final coefficient table written to: ", trip_mc_coef_file)